<a href="https://colab.research.google.com/github/cssi87m/NLP-/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=357ebaac0f36b045c2c93ea52bf50792f9e66dc98ae761841a476031b8950f32
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


Text preprocessing


In [ ]:
import autocorrect
import string
import nltk
import spacy

nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from autocorrect import Speller
from bs4 import BeautifulSoup

class TextPreprocessor():
    def __init__(self, stopwords=stopwords, stemmer=PorterStemmer()):
      self.stopwords = stopwords
      self.stemmer = stemmer
      self.spacy_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
      self.autocorrect_spell = Speller(lang='en')
    def lowercasing(self, text):
      return text.lower()
    def uppercase(self, text):
      return text.upper()
    def urlremoval(self, text):
      return " ".join([word for word in text.split() if not (word.startswith("https://") or word.startswith("http://"))])
    def tagsremoval(self, text):
      soup=BeautifulSoup(text, 'html.parser')
      return soup.get_text()
    def punctuationremoval(self, text):
      translator= text.maketrans("" , "", string.punctuation)
      return text.translate(translator)
    def spell_correct(self, text):
      return " ".join([self.autocorrect_spell.autocorrect_word(word) for word in text.split()])
    def stopwordsremoval(self, text):
      stopword=[stopwords]
      text=[word for word in text.split() if word not in stopword]
      text=" ".join([word for word in text])
      return text
    def freq_removal(self, text):
      return " ".join([word for word in text.split() if not (text.count(word)>1)])
    def stemming(self, text):
      return " ".join([self.stemmer.stem(word) for word in text.split()])
    def lemmatize(self, text):
      text=self.spacy_nlp(text)
      return " ".join([word.lemma_ for word in text])
    def preprocess(self, text,
                   lowercase=True,
                   uppercase=False,
                   urlremoval=True,
                   tagsremoval=True,
                   punctuationremoval=True,
                   spell_correct=False,
                   stopwordsremoval=True,
                   freq_removal=False,
                   stem=True,
                   lemmatize=False):
      if lowercase==True and uppercase==True:
        print("Cannot do both lowercasing and uppercasing. Please specify only one of them.")
        return
      if stem==True and lemmatize==True:
        print("Cannot do both stemming and lemmatization. Please specify only one of them.")
        return
      if lowercase==True:
        text=self.lowercasing(text)
      if uppercase==True:
        text=self.uppercase(text)
      if urlremoval==True:
        text=self.urlremoval(text)
      if tagsremoval==True:
        text=self.tagsremoval(text)
      if punctuationremoval==True:
        text=self.punctuationremoval(text)
      if spell_correct==True:
        text =self.spell_correct(text)
      if stopwordsremoval==True:
        text = self.stopwordsremoval(text)
      if freq_removal==True:
        text=self.freq_removal(text)
      if stem==True:
        text=self.stemming(text)
      if lemmatize==True:
        text=self.lemmatize(text)
      return text

preprocessor=TextPreprocessor()
exam="Hello, we are are AI Vietnam"
print(preprocessor.preprocess(exam,freq_removal=True,stem=False,lemmatize=True))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


hello we ai vietnam


2: Text Vectorization

In [ ]:
import numpy as np
import pandas as pd
class TextVectorizer():
    def __init__(self, dictionary=None):
       self.dictionary=dictionary
    def tokenize(self, text):
       return text.split()
    def create_dictionary(self, list_text):
       dictionary=[]
       for text in list_text:
          text=self.tokenize(text)
          for word in text:
             if word not in dictionary: dictionary.append(word)
       self.dictionary=dictionary
       return dictionary
    def count_vectorizer(self, text):
      if self.dictionary==None:
         return
      else:
         vector=np.zeros(len(self.dictionary))
         i=0
         for word in self.dictionary:
           if word in self.tokenize(text):
              vector[i]+=text.count(word)
           i+=1
         return [vector]
    def one_hot_vectorizer(self, text):
      if self.dictionary==None:
         return
      else:
         onehot_vector=[]
         text=self.tokenize(text)
         for word in self.dictionary:
            vector=np.zeros(len(text))
            for i in range(len(text)-1):
               if (word==text[i]): vector[i]+=1
            onehot_vector.append(vector.reshape(-1))
         return onehot_vector
vect=TextVectorizer()
text = "welcome aio 2022 2022"
vect.create_dictionary([text])
print(vect.one_hot_vectorizer(text))

[array([1., 0., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 1., 0.])]


3: Autocomplete

In [ ]:
!gdown --id 16IiXNoRJxsRBfFIr1wacrULbQHGi_lEa
!unzip txt.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16IiXNoRJxsRBfFIr1wacrULbQHGi_lEa
To: /content/txt.zip
100% 276k/276k [00:00<00:00, 70.6MB/s]
Archive:  txt.zip
   creating: txt/
 extracting: txt/B01___24_Matthew_____ENGESVN1DA_verse_34.txt  
 extracting: txt/B01___25_Matthew_____ENGESVN1DA_verse_41.txt  
 extracting: txt/B01___25_Matthew_____ENGESVN1DA_verse_6.txt  
 extracting: txt/B01___25_Matthew_____ENGESVN1DA_verse_1.txt  
 extracting: txt/B01___13_Matthew_____ENGESVN1DA_verse_48.txt  
 extracting: txt/B01___07_Matthew_____ENGESVN1DA_verse_1.txt  
 extracting: txt/B01___05_Matthew_____ENGESVN1DA_verse_44.txt  
 extracting: txt/B01___25_Matthew_____ENGESVN1DA_verse_10.txt  
 extracting: txt/B01___07_Matthew_____ENGESVN1DA_verse_18.txt  
 extracting: txt/B01___25_Mat

In [ ]:
import os
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import string

def remove_stopwords(text):
   stopword=[stopwords]
   return " ".join([word for word in text.split() if word not in stopword])
def remove_punc(text):
   return text.translate(str.maketrans("","",string.punctuation))
def dictionary(corpus_path):
   dictionary=[]
   file_name=os.listdir(corpus_path)
   dictionary=[]
   for link in file_name:
      link=os.path.join(corpus_path, link)
      link=open(link)
      link=link.read()
      link=remove_punc(remove_stopwords(link))
      for word in link.split():
         if word not in dictionary: dictionary.append(word)
   return dictionary

dictionary=dictionary("/content/txt")

i=0
text=input()
text=text.lower()
for word in dictionary:
   word_2=word.lower()
   if word_2.startswith(text):
      print(f"{i+1}.{word}")
      i+=1

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


hell
1.hell
